In [1]:
import json, re, warnings

from ast import literal_eval
from typing import Literal, Union

import numpy as np
import pandas as pd

from google import genai
from tqdm import tqdm

In [ ]:
with open("../data_input/data_dict.json", 'r', encoding="utf8") as inp:
    data_dict = json.load(inp)

In [ ]:
ORIG_DATASET = pd.read_excel("../data_input/EgeEvalDataset.xlsx")
ORIG_DATASET["distractors"] = ORIG_DATASET["distractors"].apply(literal_eval)

In [4]:
def distractors_to_labels(distractors: list[list[str]]) -> list[Literal[0, 1]]:
    labels = []

    for distractor_set, (idx, row) in zip(distractors, ORIG_DATASET.iterrows()):
        if row["question"] == "Какое высказывание СООТВЕТСТВУЕТ тексту?":
            for d in distractor_set:
                labels.append(0)
        elif row["question"] == "Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?":
            for d in distractor_set:
                labels.append(1)
        else:
            raise ValueError(row["question"])

    return labels

In [5]:
re.match("да(?![а-я])", "да.")

<re.Match object; span=(0, 2), match='да'>

In [6]:
re.match("да(?![а-я])", "данное")

In [7]:
re.match("да(?![а-я])", "да")

<re.Match object; span=(0, 2), match='да'>

In [ ]:
API_KEY = "ENTER-YOUR-API-KEY"
client = genai.Client(
    api_key=API_KEY, http_options={"base_url": "ENTER-YOUR-PROVIDER-URL"}
)

def attest_distractor_gemini(distractor: str, reading_text: str) -> Union[Literal[0, 1], None]:
    output = None
    response = client.models.generate_content(
        model="gemini-2.5-pro-preview-05-06",
        contents=f"Прочитай текст и ответь на вопрос односложно (Да или Нет). ТЕКСТ: {reading_text} ВОПРОС: ВЕРНО ЛИ СЛЕДУЮЩЕЕ УТВЕРЖДЕНИЕ? {distractor}"
    )
    result = response.text
    if re.match("да(?![а-я])", result.lower().strip()):
        output = 1
    elif re.match("нет(?![а-я])", result.lower().strip()):
        output = 0
    else:
        warnings.warn(f'Got "{result.lower().strip()}". Setting result to None')
    return output

In [9]:
0 != None

True

In [10]:
1 != None

True

In [11]:
gemini_results = []
progr = tqdm(total=sum([len(item) for k, val in data_dict.items() for item in val]))

for key, val in data_dict.items():
    val_labels = distractors_to_labels(val)
    i = 0
    for item, (_, orig_item) in zip(val, ORIG_DATASET.iterrows()):
        for distractor in item:
            gemini_results.append({
                "source": key,
                "reading_text": orig_item["reading_text"],
                "question": orig_item["question"],
                "distractor": distractor,
                "gemini_guess": int(val_labels[i] == attest_distractor_gemini(distractor, orig_item["reading_text"]))
            })
            i += 1
            progr.update()

gemini_results = pd.DataFrame(gemini_results)

  0%|          | 0/1597 [00:00<?, ?it/s]

 85%|████████▌ | 1361/1597 [3:46:55<38:42,  9.84s/it]  C:\Users\User\AppData\Local\Temp\ipykernel_13592\1632684005.py:18: UserWarning: Got "чтобы ответить "да" или "нет", мне нужно само утверждение, которое я должен оценить как верное или неверное на основе текста.

пожалуйста, предоставьте утверждение.

если предположить, что утверждение было бы, например: "иудин искренне сочувствует бедным и готов им помочь", то ответ был бы **нет**. его слова расходятся с делами (он торгуется и дает меньше денег нуждающимся), а слезы названы "крокодиловыми".

если утверждение было бы: "иудин — лицемерный человек", то ответ был бы **да**.

без самого утверждения я не могу дать однозначный ответ.". Setting result to None
  warnings.warn(f'Got "{result.lower().strip()}". Setting result to None')
100%|██████████| 1597/1597 [4:22:39<00:00,  9.49s/it]  

In [ ]:
gemini_results.to_excel("../data_output_table/gemini_results.xlsx")

In [13]:
gemini_results.groupby("source")["gemini_guess"].mean()

source
BartDG              0.500000
BartDG_ANPM         0.443114
BartDG_PM           0.461078
ChatGPT4o           0.532934
Deepseek            0.460606
MuSeRC_GPT3         0.383648
MuSeRC_T5           0.394958
RuRace_GPT3         0.391026
RuRace_T5           0.448485
true_distractors    0.957831
Name: gemini_guess, dtype: float64

In [15]:
gemini_results.groupby(["source", "question"])["gemini_guess"].mean()

source            question                                   
BartDG            Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?    0.254717
                  Какое высказывание СООТВЕТСТВУЕТ тексту?       0.933333
BartDG_ANPM       Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?    0.188679
                  Какое высказывание СООТВЕТСТВУЕТ тексту?       0.885246
BartDG_PM         Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?    0.186916
                  Какое высказывание СООТВЕТСТВУЕТ тексту?       0.950000
ChatGPT4o         Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?    0.299065
                  Какое высказывание СООТВЕТСТВУЕТ тексту?       0.950000
Deepseek          Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?    0.171429
                  Какое высказывание СООТВЕТСТВУЕТ тексту?       0.966667
MuSeRC_GPT3       Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?    0.147059
                  Какое высказывание СООТВЕТСТВУЕТ тексту?       0.807018
MuSeRC_T5         Какое высказывание НЕ СООТВЕТСТВ

In [16]:
ORIG_DATASET["question"].value_counts()

question
Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?    35
Какое высказывание СООТВЕТСТВУЕТ тексту?       20
Name: count, dtype: int64